# Chains

## Simple Sequential Chain
The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [1]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

In [49]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.1)
template = """Who is the president of the {country} ?"""
prompt_template = PromptTemplate(input_variables=["country"], template=template)
capital_chain = LLMChain(llm=llm, prompt=prompt_template)

In [50]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.1)
template = """What is the birth date of {president}?"""
prompt_template = PromptTemplate(input_variables=["president"], template=template)
bdate_chain = LLMChain(llm=llm, prompt=prompt_template)

In [51]:
overall_chain = SimpleSequentialChain(chains=[capital_chain, bdate_chain], verbose=True)

In [53]:
review = overall_chain.run("India")



> Entering new SimpleSequentialChain chain...


The current President of India is Ram Nath Kovind.


Ram Nath Kovind was born on October 1, 1945.

> Finished chain.


## SequentialChain 
A more general form of sequential chains, allowing for multiple inputs/outputs.

In [54]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

llm = OpenAI(temperature=.7)

In [73]:
template = """Could you add few lines about {topic} and {subtopic}?"""
prompt_template = PromptTemplate(input_variables=["topic", "subtopic"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [88]:
template = """Summarise the following synopsis to one line: {synopsis} """
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
summary_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="oneLine")

In [92]:

template = """Translate the following synopsis to spanish:  {synopsis}"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
hindi_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="hindisynopsis")

In [93]:
overall_chain = SequentialChain(
    chains=[synopsis_chain,hindi_chain, summary_chain],
    input_variables=["topic", "subtopic"],
    output_variables=["synopsis", "hindisynopsis", "oneLine"],
    verbose=True)

In [95]:
overall_chain({"topic":"Spain", "subtopic": "Economy"})



> Entering new SequentialChain chain...

> Finished chain.


{'topic': 'Spain',
 'subtopic': 'Economy',
 'synopsis': "\n\nSpain has a diversified and advanced economy, with services representing the largest sector (contributing around 70% of GDP), followed by industry (26%) and agriculture (4%). The country's main industries include automotive, textiles, tourism, energy, chemicals, food and beverages, electronics, telecommunications, construction, and shipbuilding. Spain also has a strong presence in the production of renewable energy, such as wind and solar power. The country is a member of the European Union, which has had a positive impact on Spain's economy, with increased investments, foreign trade, and employment opportunities. In 2020, Spain's GDP grew by 5.6%, making it one of the fastest-growing economies in the Eurozone.",
 'hindisynopsis': '\n\nEspaña tiene una economía diversificada y avanzada, con servicios que representan el sector más grande (contribuyendo alrededor del 70% del PIB), seguido por la industria (26%) y la agricultura

## LoadSummarize chain

This is the fundamental chain to perform the task of summarization over documents.

### Map reduce
The map reduce documents chain first applies an LLM chain to each document individually, treating the chain output as a new document. It then passes all the new documents to a separate combine documents chain to get a single output (the Reduce step). It can optionally first compress, or collapse, the mapped documents to make sure that they fit in the combine documents chain (which will often pass them to an LLM). This compression step is performed recursively if necessary.

This algorithm works by first splitting the entire input into small chunks using a text splitter. Then we create a summary of each of these chunks. Once we get a summary of each, the algorithm creates a summary over these summaries and provides an output

In [96]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

In [99]:
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [102]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

In [105]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)
chain({"input_documents": docs}, return_only_outputs=True)

In [111]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run( docs)

' President Biden has outlined a Unity Agenda for the Nation, which includes tackling the opioid epidemic, mental health issues, and supporting veterans. In response to Russian aggression in Ukraine, the US and its allies are taking action to seize ill-gotten gains and isolate them economically. The US is also providing military, economic, and humanitarian assistance to Ukraine. The American Rescue Plan and the Bipartisan Infrastructure Law have been passed to help Americans struggling with the pandemic and to create jobs. The President is also pushing for the passage of the Bipartisan Innovation Act, which would make record investments in emerging technologies and American manufacturing. He is encouraging the use of American products to support American jobs and is proposing a global minimum tax rate to prevent companies from avoiding taxes. Finally, he is calling for Americans to come together and move forward on COVID-19 and other issues.'

# Router
Router chain dynamically selects the next chain to use for a given input.

In [112]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [113]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [114]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [115]:
llm = OpenAI()
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [116]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [117]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [118]:
chain = MultiPromptChain(router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,)

In [120]:
print(chain.run("What is Newtons third law?"))



> Entering new MultiPromptChain chain...
physics: {'input': "What is Newton's third law?"}
> Finished chain.


Newton's third law states that for every action, there is an equal and opposite reaction. In other words, the forces between two objects always come in pairs, and these forces are equal in magnitude and opposite in direction.


In [121]:
print(chain.run("What is the largest 3 digit prime number?"))



> Entering new MultiPromptChain chain...
math: {'input': 'What is the largest three digit prime number?'}
> Finished chain.


The largest three digit prime number is 997.


In [126]:
print(chain.run("Where did the First World War take place?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'Where did the First World War take place?'}
> Finished chain.


I do not know the answer to this question, as it is outside the scope of my expertise in physics.
